In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("catalogo","catalog_comercial")
dbutils.widgets.text("container","bronze")
dbutils.widgets.text("esquema","silver")
dbutils.widgets.text("storage","sajhsvdesa01")

In [0]:
container=dbutils.widgets.get("container")
esquema=dbutils.widgets.get("esquema")
catalogo=dbutils.widgets.get("catalogo")
storage=dbutils.widgets.get("storage")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import functions as F


# TABLAS MAESTRAS COMERCIAL DIGITAL

In [0]:
## leer tablas maestras en container bronze
df_clientes_tr = spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/cliente")
df_ejecutivo_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/ejecutivo")
df_ejecutivo_digital_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/ejecutivo_digital")
df_medio_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/medio")
df_tipo_medio_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/tipo_medio")
df_producto_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/producto")
df_tipo_producto_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/tipo_producto")
df_grupo_laboral_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/grupo_laboral")
df_tipo_venta_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/tipo_venta")
df_cobertura_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/cobertura")
df_condicion_pago_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/condicion_pago")
df_forma_factura_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/forma_factura")
df_contrato_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/contrato")

In [0]:
# Clientes
df_clientes_sil=df_clientes_tr.select(
    col("C_CLIENTE"),
    col("C_RAZON_SOCIAL"),
    col("C_DIRECCION"),
    col("C_PAGINA_WEB"),
    col("C_CORREO_EMPRESA"))
df_clientes_sil = df_clientes_sil.withColumn("C_PAGINA_WEB", 
                                        when((col("C_PAGINA_WEB").isNull() | (col("C_PAGINA_WEB")=="") | (col("C_PAGINA_WEB")=="http://")), lit("web_nr")).otherwise(col("C_PAGINA_WEB")))\
                                        .withColumn("C_CORREO_EMPRESA", when((col("C_CORREO_EMPRESA").isNull() | (col("C_CORREO_EMPRESA")=="") | (col("C_CORREO_EMPRESA")=="@")), lit("correo_nr")).otherwise(col("C_CORREO_EMPRESA")))\
                                        .withColumn("ingestion_date", current_timestamp())

df_clientes_sil=df_clientes_sil.withColumnRenamed("C_CLIENTE","cod_cliente")\
            .withColumnRenamed("C_RAZON_SOCIAL","Razon_Social")\
            .withColumnRenamed("C_DIRECCION","Direccion")\
            .withColumnRenamed("C_PAGINA_WEB","Pagina_web")\
            .withColumnRenamed("C_CORREO_EMPRESA","Correo_Empresa")\
            .withColumn("ingestion_date",current_timestamp()
        )

In [0]:
# ejecutivo
df_ejecutivo_sil=df_ejecutivo_tr.select(col("C_EJECUTIVO").alias("cod_ejecutivo")
                                        ,concat(col("C_NOMBRES"),lit(" "),col("C_APELLIDOS")).alias("nombre_ejecutivo"),
                                        col("C_GRUPO_LABORAL").alias("cod_grupo_laboral"),
                                        col("C_USUARIO_EJECUTIVO_PADRE")
                                        ).withColumn("ingestion_date",current_timestamp())                                      


In [0]:
df_ejecutivo_digital_sil=df_ejecutivo_digital_tr.select(col("N_EJECUTIVO_DIGITAL").alias("sk_ejecutivo_digital"),
                                                       col("C_EJECUTIVO").alias("cod_ejecutivo"),
                                                       col("C_EJECUTIVO_DIGITAL").alias("cod_ejecutivo_digital"),
                                                       col("C_ACTIVO").alias("activo")).withColumn("ingestion_date",current_timestamp())
 			

In [0]:
df_medio_sil=df_medio_tr.select(col("C_MEDIO").alias("cod_medio"),
                               col("C_DESCRIPCION").alias("nombre_medio"),
                               col("C_TIPO_MEDIO").alias("cod_tipo_medio")
                               ).withColumn("ingestion_date",current_timestamp())

In [0]:
df_tipo_medio_sil=df_tipo_medio_tr.select(col("C_TIPO_MEDIO").alias("cod_tipo_medio"),
                                 col("C_DESCRIPCION").alias("nombre_tipo_medio")
                                 ).withColumn("ingestion_date",current_timestamp())

In [0]:
df_producto_sil=df_producto_tr.select(col("C_MEDIO").alias("cod_medio"),
                                     col("C_PRODUCTO_MEDIO").alias("cod_producto"),
                                     col("C_NOMBRE_PRODUCTO").alias("nombre_producto"),
                                     col("C_TIPO_PRODUCTO").alias("cod_tipo_producto")).withColumn("ingestion_date",current_timestamp())


In [0]:
df_tipo_producto_sil=df_tipo_producto_tr.select(col("C_TIPO_PRODUCTO").alias("cod_tipo_producto"),
                                     col("C_DESCRIPCION").alias("nombre_tipo_producto"),
                                        ).withColumn("ingestion_date",current_timestamp())


In [0]:
df_grupo_laboral_sil=df_grupo_laboral_tr.select(col("C_GRUPO_LABORAL").alias("cod_grupo_laboral"),
                                               col("C_NOMBRE_GRUPO").alias("nombre_grupo_laboral")
                                               ).withColumn("ingestion_date",current_timestamp())


In [0]:
df_cobertura_sil=df_cobertura_tr.select(col("C_MEDIO").alias("cod_medio"),
                                       col("C_COBERTURA").alias("cod_cobertura"),
                                       col("C_DESCRIPCION").alias("nombre_cobertura")).withColumn("ingestion_date",current_timestamp())
               


In [0]:
df_condicion_pago_sil=df_condicion_pago_tr.select(col("C_CONDICION_PAGO").alias("cod_condicion_pago"),
                                                 col("C_DESCRIPCION").alias("condicion_pago")).withColumn("ingestion_date",current_timestamp())


In [0]:
df_forma_factura_sil=df_forma_factura_tr.select(col("C_FORMA_FACTURA").alias("cod_forma_factura"),
                                                 col("C_FORMA_FACTURA_EFACT").alias("forma_pago"),
                                                 col("C_DESCRIPCION").alias("descripcion")).withColumn("ingestion_date",current_timestamp())


In [0]:
df_contrato_sil=df_contrato_tr.select(col("C_CONTRATO").alias("cod_contrato"),
                                     col("C_CLIENTE_FK_CLI").alias("cod_clie_fk"),
                                     col("N_BRUTO_S").alias("bruto_soles"),
                                     col("N_BRUTO_D").alias("bruto_dolares"),
                                     col("N_DESCUENTO_AGENCIA_D").alias("descuento_agencia_soles"),
                                     col("N_DESCUENTO_AGENCIA_S").alias("descuento_agencia_dolares"),
                                     col("N_NETO_S").alias("neto_soles"),
                                     col("N_NETO_D").alias("neto_dolares"),
                                     col("N_IGV_S").alias("igv_soles"),
                                     col("N_IGV_D").alias("igv_dolares")).withColumn("ingestion_date",current_timestamp())



# TRANSACCIONALES

In [0]:
df_pedido_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/pedido")
df_pedido_detalle_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/pedido_detalle")
df_orden_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/orden")
df_orden_detalle_tr=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/orden_detalle")

In [0]:


df_pedido_sil=df_pedido_tr.select(col("N_PEDIDO").alias("cod_pedido"),
                                 col("D_INICIO").alias("fecha_inicio"),
                                 col("D_FIN").alias("fecha_fin"),
                                 col("C_CLIENTE").alias("cod_cliente"),
                                 col("C_CONTRATO").alias("cod_contrato"),
                                 col("C_MONEDA").alias("moneda"),
                                 col("N_TIPO_CAMBIO").alias("cod_tipo_cambio"),
                                 col("C_AGENCIA").alias("cod_agencia"),
                                 col("C_EJECUTIVO").alias("cod_ejecutivo"),
                                 col("C_MEDIO").alias("cod_medio"),
                                 col("N_TOTAL_S").alias("total_soles"),
                                 col("C_FORMA_FACTURA").alias("cod_forma_factura"),
                                 col("C_CONDICION_PAGO").alias("cod_condicion_pago")
                                 ).withColumn("ingestion_date",current_timestamp())

## transform fecha_inicio, fecha_fin con to_date
df_pedido_sil=df_pedido_sil.withColumn("fecha_inicio",F.to_date(col("fecha_inicio"),"YYYY-MM-dd"))
df_pedido_sil=df_pedido_sil.withColumn("fecha_fin",F.to_date(col("fecha_fin"),"YYYY-MM-dd"))                                 

In [0]:
df_pedido_detalle_sil=df_pedido_detalle_tr.select(col("N_PEDIDO").alias("cod_pedido"),
                                                 col("N_EPDIDO_DETALLE").alias("cod_pedido_detalle"),
                                                 col("C_MEDIO").alias("cod_medio"),
                                                 col("C_PRODUCTO_MEDIO").alias("cod_producto"),
                                                 col("C_COBERTURA").alias("cod_cobertura"),
                                                 col("C_TIPO_COMPONENTE").alias("cod_tipo_componente"),
                                                 col("C_CORTE").alias("cod_corte"),
                                                 col("C_EJECUTIVO").alias("cod_ejecutivo"),
                                                 col("N_TOTAL_S").alias("total_soles"),
                                                 col("N_TOTAL_D").alias("total_dolares"),
                                                 col("N_CATEGORIA_DIGITAL").alias("cod_categoria_digital")).withColumn("ingestion_date",current_timestamp())


In [0]:
df_orden_sil=df_orden_tr.select(col("C_ORDEN").alias("cod_orden"),
                               col("D_INICIO").alias("fecha_inicio"),
                               col("D_FIN").alias("fecha_fin"),
                               col("C_MEDIO").alias("cod_medio"),
                               col("C_PRODUCTO_MEDIO").alias("cod_producto"),
                               col("C_COBERTURA").alias("cod_cobertura"),
                               col("C_EJECUTIVO").alias("cod_ejecutivo"),
                               col("N_MONTO_S").alias("monto_soles"),
                               col("N_MONTO_D").alias("monto_dolares"),
                               col("N_MONTO_NETO_S").alias("monto_neto_soles"),     
                               col("N_MONTO_NETO_D").alias("monto_neto_dolares"),     
                               col("N_TIPO_CAMBIO").alias("tipo_cambio"),     
                               col("N_PORCENTAJE_APAP").alias("porcentaje_apap"),
                               col("N_PEDIDO").alias("cod_pedido"),
                               col("C_PAUTABLE").alias("es_pautable")).withColumn("ingestion_date",current_timestamp())

## transform fecha_inicio, fecha_fin con to_date
df_orden_sil=df_orden_sil.withColumn("fecha_inicio",F.to_date(col("fecha_inicio"),"YYYY-MM-dd"))
df_orden_sil=df_orden_sil.withColumn("fecha_fin",F.to_date(col("fecha_fin"),"YYYY-MM-dd"))    
                            

In [0]:
df_orden_detalle_sil=df_orden_detalle_tr.select(col("N_PEDIDO").alias("cod_pedido"),
                                               col("N_PEDIDO_DETALLE").alias("cod_pedido_detalle"),
                                               col("C_ORDEN").alias("cod_orden"),
                                               col("N_CANTIDAD_TOTAL").alias("cantidad_total"),
                                               col("N_MONTO_S").alias("monto_soles"),
                                               col("N_MONTO_D").alias("monto_dolares"),
                                               col("N_MONTO_NETO_S").alias("monto_neto_soles"),     
                                               col("N_MONTO_NETO_D").alias("monto_neto_dolares"),     
                                               col("N_MONTO_REAL_S").alias("monto_real_soles"),
                                               col("N_MONTO_REAL_D").alias("monto_real_dolares")).withColumn("ingestion_date",current_timestamp())


In [0]:
df_clientes_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.cliente")
df_ejecutivo_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.ejecutivo")
df_cobertura_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.cobertura")
df_medio_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.medio")
df_tipo_medio_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.tipo_medio")
df_producto_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.producto")
df_tipo_producto_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.tipo_producto")
df_grupo_laboral_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.grupo_laboral")
df_ejecutivo_digital_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.ejecutivo_digital")
df_contrato_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.contrato")
df_forma_factura_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.forma_factura")
df_condicion_pago_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.condicion_pago")
df_pedido_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.pedido")
df_pedido_detalle_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.pedido_detalle")
df_orden_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.orden")
df_orden_detalle_sil.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.orden_detalle")


